<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

### pandas 라이브러리와 탐색적 데이터 분석 과정 익히기

> 다양한 데이터 분석 케이스를 통해 데이터 분석과 pandas 라이브러리 활용에 대해 익히보기로 합니다.

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보

### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- 평균 거래액은 얼마일까?
- 일별, 주별, 월별 판매 트렌드는?
- **어떤 카테고리가 가장 많은 상품이 팔렸을까?**
- 평균 배송 시간은?
</div>

### 1. 카테고리별 판매 분석

In [ ]:
import pandas as pd
PATH = "00_data/"

In [ ]:
products = pd.read_csv(PATH + "olist_products_dataset.csv", encoding='utf-8-sig')
customers = pd.read_csv(PATH + "olist_customers_dataset.csv", encoding='utf-8-sig')
geolocation = pd.read_csv(PATH + "olist_geolocation_dataset.csv", encoding='utf-8-sig')
order_items = pd.read_csv(PATH + "olist_order_items_dataset.csv", encoding='utf-8-sig')
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
reviews = pd.read_csv(PATH + "olist_order_reviews_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
sellers = pd.read_csv(PATH + "olist_sellers_dataset.csv", encoding='utf-8-sig')
category_name = pd.read_csv(PATH + "product_category_name_translation.csv", encoding='utf-8-sig')

#### products 와 category_name 합치기

In [ ]:
products.head()

In [ ]:
category_name.head()

In [ ]:
products_category = pd.merge(products, category_name, on='product_category_name')

In [ ]:
products_category

#### 필요한 컬럼만 가져오기

In [ ]:
products_category_only = products_category[['product_id', 'product_category_name_english']].copy()
products_category_only.head()

In [ ]:
order_items.isnull().sum()

In [ ]:
order_items.head()

In [ ]:
products_category_order = pd.merge(products_category_only, order_items, on='product_id')
products_category_order.head()

### 2. 카테고리별 거래 건수 확인하기
1. 카테고리 이름으로 grouping 한다: products_category_order.groupby('product_category_name_english')
2. grouping 할 때 order는 거래액이 아니라 거래건수로 count 를 한다: .aggregate({'order_id':'count'})
   - .aggregate() 또는 .agg() 를 사용
2. 거래건수로 count 한 값의 컬럼명을 order_count 로 바꾼다: .rename(columns={'order_id':'order_count'})

### 참고: aggregate 함수

In [ ]:
import pandas as pd
df = pd.DataFrame({
    "미국": [2.1, 2.2, 2.3],
    "한국": [0.4, 0.5, 0.45],
    "중국": [10, 13, 15]    
})
df

#### 다양한 함수 사용하기
- 주요 함수: sum, min, max, mean, std, count 등

In [ ]:
df.agg(['sum', 'min'])

#### 특정 컬럼만 함수 적용해서 가져오기

In [ ]:
df.agg({'미국' : 'count', '한국': 'sum'})

#### 행에 대해 함수 적용하기

In [ ]:
df.agg("mean", axis="columns")

#### 특정 컬럼만 grouping 하고, 함수 적용해서 가져오기

In [ ]:
df = pd.DataFrame({
    "년도": [2000, 2010, 2020, 2000, 2000, 2020],    
    "미국": [2.1, 2.2, 2.3, 3.1, 4.1, 5.1],
    "한국": [0.4, 0.5, 0.45, 0.7, 0.9, 1.2],
    "중국": [10, 13, 15, 12, 45, 56]    
})
df

In [ ]:
df.groupby('년도').sum()

In [ ]:
df.groupby('년도').agg({'미국':'sum', '한국':'mean'})

### 카테고리별 거래 건수 확인하기

In [ ]:
products_category_order = products_category_order.groupby('product_category_name_english').aggregate({'order_id':'count'}).rename(columns={'order_id':'order_count'})

In [ ]:
products_category_order.head()

#### Plotly Express 기반 그래프 그리기

In [ ]:
import plotly.express as px

# products_category_order: 인덱스에 제품 카테고리, 'order_count' 열에 주문 수가 저장되어 있음
fig = px.bar(
    x=products_category_order.index,              # x축: 제품 카테고리 (인덱스 사용)
    y=products_category_order['order_count'],       # y축: 주문 수 (order_count 열)
    title="Order Count by Product Category",        # 그래프 제목
    labels={
        'x': 'Product Category',                    # x축 레이블 지정
        'y': 'Order Count'                          # y축 레이블 지정
    }
)

# x축의 제품 카테고리 이름이 길 경우 겹치지 않도록 라벨 각도 조정 (선택 사항)
fig.update_layout(xaxis_tickangle=-45)

# 그래프 출력
fig.show()


### plotly Ficture 객체로 그래프 그려보기

#### 카테고리별 거래건수로 정렬하기

In [ ]:
products_category_order = products_category_order.sort_values(by='order_count',ascending=False)
products_category_order.head()

In [ ]:
import plotly.graph_objects as go  # Plotly의 그래프 객체 모듈을 임포트합니다.

# 새 Figure 객체를 생성합니다.
fig = go.Figure()

# 막대 그래프(Bar Chart) trace를 추가합니다.
fig.add_trace(
    go.Bar(
        x=products_category_order.index,      # x축: 제품 카테고리 (DataFrame의 인덱스)
        y=products_category_order['order_count']  # y축: 각 카테고리의 주문 수 (order_count 컬럼)
    )
)

# 그래프의 레이아웃(제목, 축, 템플릿 등)을 업데이트합니다.
fig.update_layout(
    {
        "title": {
            "text": "<b>The number of Order per Category in Brazilian Olist E-Commerce company</b>",  # 그래프 제목 (HTML 태그를 사용해 굵게 표시)
            "x": 0.5,   # 제목을 수평 중앙에 위치 (0.5는 중앙을 의미)
            "y": 0.9,   # 제목의 수직 위치를 0.9로 설정 (상단에 가까운 위치)
            "font": {
                "size": 15  # 제목의 글자 크기를 15로 설정
            }
        },
        "xaxis": {
            "title": "from Oct. 2016 to Sep. 2018",  # x축 제목 (기간 정보)
            "showticklabels": True,                  # x축 눈금 라벨 표시 여부를 True로 설정
            "tickfont": {
                "size": 7  # x축 눈금 라벨의 글자 크기를 7로 설정
            }
        },
        "yaxis": {
            "title": "The number of Order"  # y축 제목 (주문 수)
        },
        "template": 'plotly_white'  # 그래프의 전체 템플릿을 'plotly_white' (흰색 배경)로 설정
    }
)

# 최종 그래프를 화면에 출력합니다.
fig.show()


#### 주요 카테고리만 확인하기

In [ ]:
products_category_order = products_category_order[products_category_order['order_count'] > 1000]
products_category_order.head()

In [ ]:
import plotly.graph_objects as go  # Plotly의 그래프 객체 모듈을 불러옵니다.

# 새로운 Figure 객체를 생성합니다.
fig = go.Figure()

# 막대 그래프(Bar Chart)를 추가합니다.
fig.add_trace(
    go.Bar(
        x=products_category_order.index,               # x축: 제품 카테고리 (DataFrame의 인덱스)
        y=products_category_order['order_count']         # y축: 각 카테고리의 주문 수 ('order_count' 컬럼)
    )
)

# 그래프의 레이아웃을 업데이트합니다.
fig.update_layout(
    {
        "title": {
            "text": "<b>The number of Order per Category in Brazilian Olist E-Commerce company</b>",  # 그래프 제목 (HTML 태그로 굵게 표시)
            "x": 0.5,    # 제목의 x 위치를 0.5로 설정하여 중앙에 배치
            "y": 0.9,    # 제목의 y 위치를 0.9로 설정 (그래프 상단에 가깝게 배치)
            "font": {
                "size": 15  # 제목 글꼴 크기를 15로 지정
            }
        },
        "xaxis": {
            "title": "from Oct. 2016 to Sep. 2018",  # x축 제목: 기간 정보
            "tickfont": {
                "size": 7  # x축 눈금 라벨의 글꼴 크기를 7로 설정
            }
        },
        "yaxis": {
            "title": "The number of Order"  # y축 제목: 주문 수
        },
        "template": 'plotly_white'  # 그래프의 전체 템플릿을 'plotly_white'로 설정 (흰색 배경)
    }
)

# 완성된 그래프를 화면에 표시합니다.
fig.show()


<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>